In [1]:
import pandas as pd 
import numpy as np
import re 

In [2]:
data = pd.read_csv('DL_CQI_TA_.csv')

In [3]:
listCol = data.columns.array.to_numpy().tolist()
listCol

['Date',
 'eNodeB Name',
 'eNodeB Function Name',
 'Cell Name',
 'LocalCell Id',
 'Cell FDD TDD Indication',
 'Integrity',
 'L.ChMeas.CQI.DL.0',
 'L.ChMeas.CQI.DL.1',
 'L.ChMeas.CQI.DL.10',
 'L.ChMeas.CQI.DL.11',
 'L.ChMeas.CQI.DL.12',
 'L.ChMeas.CQI.DL.13',
 'L.ChMeas.CQI.DL.14',
 'L.ChMeas.CQI.DL.15',
 'L.ChMeas.CQI.DL.2',
 'L.ChMeas.CQI.DL.3',
 'L.ChMeas.CQI.DL.4',
 'L.ChMeas.CQI.DL.5',
 'L.ChMeas.CQI.DL.6',
 'L.ChMeas.CQI.DL.7',
 'L.ChMeas.CQI.DL.8',
 'L.ChMeas.CQI.DL.9',
 'L.RA.TA.UE.Index0',
 'L.RA.TA.UE.Index1',
 'L.RA.TA.UE.Index10',
 'L.RA.TA.UE.Index11',
 'L.RA.TA.UE.Index2',
 'L.RA.TA.UE.Index3',
 'L.RA.TA.UE.Index4',
 'L.RA.TA.UE.Index5',
 'L.RA.TA.UE.Index6',
 'L.RA.TA.UE.Index7',
 'L.RA.TA.UE.Index8',
 'L.RA.TA.UE.Index9']

In [56]:

col={'eNodeB_Name' :data['eNodeB Name'],'eNodeB_Function_Name' :data['eNodeB Function Name'],'Cell_Name' :data['Cell Name'],'LocalCell_Id' :data['LocalCell Id']}
df2=pd.DataFrame( col )
for x in listCol[23:]:
    df2[x] = data[x]

In [57]:
listCol_df2 = df2.columns.array.to_numpy().tolist()
listCol_df2

['eNodeB_Name',
 'eNodeB_Function_Name',
 'Cell_Name',
 'LocalCell_Id',
 'L.RA.TA.UE.Index0',
 'L.RA.TA.UE.Index1',
 'L.RA.TA.UE.Index10',
 'L.RA.TA.UE.Index11',
 'L.RA.TA.UE.Index2',
 'L.RA.TA.UE.Index3',
 'L.RA.TA.UE.Index4',
 'L.RA.TA.UE.Index5',
 'L.RA.TA.UE.Index6',
 'L.RA.TA.UE.Index7',
 'L.RA.TA.UE.Index8',
 'L.RA.TA.UE.Index9']

In [62]:
#print(df2.iat[0,5])
#print(type (df2.iat[0,5])) 

In [58]:
for x in listCol_df2[4:]:
    df2[x] = df2[x].astype(int)

In [59]:
#drop rows of zeros
df3=df2.loc[~(df2[listCol_df2[4:]]==0).all(axis=1)] 

In [60]:
#drop columns of zeros
df3 = df3.loc[:, (df3 != 0).any(axis=0)]

In [61]:
df3.to_csv('counters_only.csv', index=False, index_label=False)

In [13]:
data2 = pd.read_csv('counters_only.csv')

In [14]:
data2["sum"] = data2.drop(['eNodeB_Name','eNodeB_Function_Name','Cell_Name','LocalCell_Id',], axis=1,inplace=False).sum(axis=1)

In [15]:
data2.to_csv('CountersWithSum.csv', index=False, index_label=False)

In [41]:
data3 = pd.read_csv('CountersWithSum.csv')

In [42]:
data3 = data3.set_index(['eNodeB_Name','eNodeB_Function_Name','Cell_Name','LocalCell_Id',])
data3 = data3.div(data3['sum'], axis=0)

In [43]:
data3.to_csv('percentage.csv', index=False, index_label=False)


In [44]:
data3.drop(['sum'], axis=1,inplace=True)

In [45]:
#loop through all cell and get overshooting counters
def getCounterAtOvershoot(dataset, dataWithcells):
    sum = 0 
    listOfCounters = { 'NameOfEnodB':[],'FunctionName-Of-EnodB':[],'NameOfCell':[],'ID-OfLocalCell':[],'MaxCounter':[],}
    for i in range(0, 32869):
        for col in dataset.columns:
            sum = sum + dataset.iloc[i][col]
            if sum >= 0.8:
                listOfCounters['NameOfEnodB'].append(dataWithcells.iloc[i]['eNodeB_Name'])
                listOfCounters['FunctionName-Of-EnodB'].append(dataWithcells.iloc[i]['eNodeB_Function_Name'])
                listOfCounters['NameOfCell'].append(dataWithcells.iloc[i]['Cell_Name'])
                listOfCounters['ID-OfLocalCell'].append(dataWithcells.iloc[i]['LocalCell_Id'])
                listOfCounters['MaxCounter'].append(col)
                sum = 0
                break 
    return listOfCounters

In [46]:
c = getCounterAtOvershoot(data3,data2)

In [47]:
dataWithMaxCounter= pd.DataFrame.from_dict(c)

In [48]:
dataWithMaxCounter.to_csv('CellsWithMaxCounter.csv', index=False, index_label=False)

In [49]:
listOfCounters={
 'L.RA.TA.UE.Index0':[0,156],
 'L.RA.TA.UE.Index1':[156,312],
 'L.RA.TA.UE.Index2':[312,624],
 'L.RA.TA.UE.Index3':[624,1092],
 'L.RA.TA.UE.Index4':[1092,2028],
 'L.RA.TA.UE.Index5':[2028,3588],
 'L.RA.TA.UE.Index6':[3588,6630],
 'L.RA.TA.UE.Index7':[6708,14508],
 'L.RA.TA.UE.Index8':[14508,30108],
 'L.RA.TA.UE.Index9':[30108,53508],
 'L.RA.TA.UE.Index10':[53508,76908],
 'L.RA.TA.UE.Index11':[76908],
}

In [50]:
CELLS_WITH_COUNTERS = pd.read_csv('CellsWithMaxCounter.csv')

In [55]:
def getDistance(SetWithCounters): 
    list1 = []
    list2 = []
    for i in range(0, 32869): 
        for key , value in listOfCounters.items(): 
            if key == SetWithCounters.iloc[i]['MaxCounter']:
                list1.append(value[0])
                list2.append(value[1])
    SetWithCounters.insert(5, 'MinDistance', list1, True)
    SetWithCounters.insert(6, 'MaxDistance', list2, True)
    return SetWithCounters 

In [56]:
CELLS_WITH_DISTANCE = getDistance(CELLS_WITH_COUNTERS)

In [57]:
CELLS_WITH_DISTANCE.to_csv('CELLS_WITH_DISTANCE_Range.csv', index=False, index_label=False)

In [2]:
###############################################################################################################
####################--------MERGE_site_to_siteWith_CELLS_WITH_DISTANCE_Range----------#########################
###############################################################################################################
df1= pd.read_csv('CELLS_WITH_DISTANCE_Range.csv')
df2 = pd.read_csv('Site_To_Site_Distance.csv')


In [4]:
##-->drop_cell_which_has_[AvgSiteToSiteDistance==0]<--##
df2 = df2[(df2[['AvgSiteToSiteDistance']] != 0).all(axis=1)]

In [8]:
new_df=df1.merge( df2.set_index('NameOfCell'), on='NameOfCell')

In [18]:
new_df = new_df.T.drop_duplicates().T

In [19]:
new_df.to_csv('AfterMerge.csv', index=False, index_label=False)

In [2]:
#########################################################
df3= pd.read_csv('AfterMerge.csv')

In [3]:
#loop to GET_OVERSHOOTING_Indicator
def getOvershootIndicator(dataset):
    MinOvershoot_Idicator=[]
    MaxOvershoot_Idicator=[]
    for i in range(0, 32697):
        MinOvershoot_Idicator.append(dataset['MinDistance'][i]/ ((dataset['AvgSiteToSiteDistance'][i])*1000))
        MaxOvershoot_Idicator.append(dataset['MaxDistance'][i]/((dataset['AvgSiteToSiteDistance'][i])*1000))
               
    dataset.insert(9, 'MinOvershoot_Idicator', MinOvershoot_Idicator, True)
    dataset.insert(10, 'MaxOvershoot_Idicator', MaxOvershoot_Idicator, True)
    return dataset 

In [4]:
Indicators=getOvershootIndicator(df3)

In [6]:
############################################################################################################
###############------------->Loop to Compare_OVERSHOOTING_Indicator_With_Threshold ==2 <----------###########
############################################################################################################
def getOvershootingCells(dataset):
    #MIN_Overshooting=[]
    MAX_Overshooting=[]
    for i in range(0, 32697):
        #if (dataset['MinOvershoot_Idicator'][i]>= dataset['DISTANCE_Threshold'][i]):
        #    MIN_Overshooting.append('YES_overshooted')
        #else:
        #    MIN_Overshooting.append('Not_overshooted')
        if (dataset['MaxOvershoot_Idicator'][i]>= 2):
            MAX_Overshooting.append('YES_overshooted')
        else:   
            MAX_Overshooting.append('NOT_overshooted')

    #dataset.insert(11, 'OvershootingOnMinIdicator', MIN_Overshooting, True)
    dataset.insert(11, 'OvershootingOnMaxIdicator', MAX_Overshooting, True)
    return dataset 

In [7]:
overshooting=getOvershootingCells(df3)

In [8]:
############################################################################################################
###############------------->IF_RURAL_compare_Footrrint_With_Threshold = 5km<----------#################################
###############------------->IF_dense_compare_Footprint_With_Threshold = 1km<----------#################################
############################################################################################################
def getOvershootingCells2(dataset):
    MAX_Overshooting=[]
    for i in range(0, 32697):
        if (dataset['MaxDistance'][i]>= dataset['DISTANCE_Threshold'][i]*1000): #MaxDistance in [m] && DISTANCE_Threshold in [Km]
            MAX_Overshooting.append('YES_overshooted')
        else:   
            MAX_Overshooting.append('NOT_overshooted')

    dataset.insert(11, 'OvershootingOnDISTANCE_Threshold', MAX_Overshooting, True)
    return dataset 

In [10]:
overshooting2=getOvershootingCells2(df3)

In [11]:
overshooting.to_csv('OvershootingCells.csv', index=False, index_label=False)